In [6]:
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf
import glob
import shutil
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [5]:
import os
os.chdir(r"/Users/priyankadwivedi/Documents/Kaggle/CatvsDogs/train")

In [3]:
#Take all images for cats and dogs and resize to 227, 227. Keep color of image
size = 227, 227
for infile in glob.glob("/Users/priyankadwivedi/Documents/Kaggle/CatvsDogs/train/*.jpg"):
    outfile = os.path.splitext(infile)[0] + ".small"    
    file, ext = os.path.splitext(infile)
    im = Image.open(infile)
    out = im.resize((size), Image.ANTIALIAS)
    out.save(outfile, "JPEG")

In [7]:
#Display reduced gray scale images
im = Image.open("cat.250.small")
print im.format, im.size, im.mode
im.show()

JPEG (227, 227) RGB


In [4]:
#Display reduced gray scale images
im = Image.open("dog.1500.small")
print im.format, im.size, im.mode
im.show()

IOError: [Errno 2] No such file or directory: 'dog.1500.small'

In [8]:
import numpy as np
image_size = 227
pixel_depth = 255
image_files = 25000
num_channels = 3
dataset = np.ndarray(shape= (image_files, image_size, image_size, num_channels), dtype= np.float32)
target = np.ndarray(shape= (image_files), dtype= np.int_)
num_images = 0
for filename in glob.glob("/Users/priyankadwivedi/Documents/Kaggle/CatvsDogs/train/*.small"):                         
  
  if num_images%5000 == 0: print(num_images)
  try:
      #image_data = (ndimage.imread(filename, flatten = True).astype(float)) 
      image_data = (ndimage.imread(filename, flatten = False).astype(float) - pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size, num_channels):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      name = os.path.basename(filename)
      if name.split(".")[0] == "dog":
          target[num_images] = 1
      else:
          target[num_images] = 0
      num_images = num_images + 1
  except IOError as e:
      print('Could not read:', filename, ':', e, '- it\'s ok, skipping.')

0
5000
10000
15000
20000


In [ ]:
print('Dataset shape:', dataset.shape)
#print('Dataset Mean:', np.mean(dataset))
#print('Dataset Standard deviation:', np.std(dataset))
print('Dataset Max:', np.amax(dataset))
print('Dataset Min:', np.amin(dataset))
print('Target shape:', target.shape)
print('Target Mean:', np.mean(target))
print('Target Standard deviation:', np.std(target))
print('Target Max:', np.amax(target))
print('Target Min:', np.amin(target))

In [ ]:
#Pickle to be safe
# Pickle again
os.chdir(r"/Users/priyankadwivedi/Desktop/tensor/P5")
pickle_file = 'catdog227_temp.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'dataset': dataset,
    'target': target,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
#Randomize dataset and target
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
all_dataset, all_labels = randomize(dataset, target)

In [ ]:
#Display images and check for randomization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
n=12605
image_array = (all_dataset[n])
image_array.shape

plt.imshow(image_array, cmap='viridis', interpolation='None')
print(all_labels[n])

In [ ]:
# split the full dataset of 25k images into train - 20k images and test - 5k images 
from sklearn import cross_validation 
X_train, X_valid, y_train, y_valid = cross_validation.train_test_split(
all_dataset, all_labels, test_size=0.2, random_state=252)
print("train dataset", X_train.shape, y_train.shape)
print("Validation dataset", X_valid.shape, y_valid.shape)

In [ ]:
#Check if split has happened properly
n=2900
image_array = (X_train[n])
image_array.shape

plt.imshow(image_array, cmap='Greys', interpolation='None')
print(y_train[n])

In [ ]:
#Split training again into test dataset
from sklearn import cross_validation 
X_valid, X_test, y_valid, y_test = cross_validation.train_test_split(
X_valid, y_valid, test_size=0.2, random_state=233)
print("valid dataset", X_valid.shape, y_valid.shape)
print("test dataset", X_test.shape, y_test.shape)

In [ ]:
# Resize training and test dataset
image_size = 227
num_labels = 2
num_channels = 3 # grayscale
def reshape(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset_full, train_labels_full = reshape(X_train, y_train)
valid_dataset_full, valid_labels_full = reshape(X_valid, y_valid)
test_dataset_full, test_labels_full = reshape(X_test, y_test)
print('Training set', train_dataset_full.shape, train_labels_full.shape)
print('Validation set', valid_dataset_full.shape, valid_labels_full.shape)
print('Testing set', test_dataset_full.shape, test_labels_full.shape)

In [ ]:
# Pickle again
os.chdir(r"/Users/priyankadwivedi/Desktop/tensor/P5")
pickle_file = 'catdog227.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset_full': train_dataset_full,
    'train_labels_full': train_labels_full,
    'valid_dataset_full': valid_dataset_full,
    'valid_labels_full': valid_labels_full,
    'test_dataset_full': test_dataset_full,
    'test_labels_full': test_labels_full,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise